In [41]:
from sympy import *
from IPython.display import display
import numpy as np


a,L1,L2,L3,d,D,m2,mr,I1,I2 = symbols("alpha,L_1,L_2,L_3,d,D,m_2,m_r,I_1,I_2")

external_inputs_num = 3
M1,M2,Fr = symbols("tau_1 tau_2 F_r")

constrain_Eq_num = 3
lan1,lan2,lan3 = symbols("lambda_1 lambda_2 lambda_3")


# I1=m1*L1**2/3  #momet of Inertia of L1 about A
# I2=m2*L2**2/12 #about G

t = Symbol('t')

x = Function("delta_1")(t) 
y = Function("phi_1")(t)
z = Function("delta_2")(t) 
w = Function("phi_2")(t)

xd = Function("\dot{\\delta}_1")(t)
yd = Function("\dot{\\phi}_1")(t)
zd = Function("\dot{\\delta}_2")(t)
wd = Function("\dot{\\phi}_2")(t)

xdd = Function("\ddot{\\delta}_1")(t)
ydd = Function("\ddot{\\phi}_1")(t)
zdd = Function("\ddot{\\delta}_2")(t)
wdd = Function("\ddot{\\phi}_2")(t)


In [42]:

r_G1A = Matrix([  -L1*cos(x+a)-L2*sin(y)/2  ,   -L1*sin(x+a)-L2*cos(y)/2    ])
r_G2B = Matrix([  -L1*cos(z-a)-L2*sin(w)/2  , -D+L1*sin(z-a)+L2*cos(w)/2    ])        
r_Gr  = Matrix([  -d                      ,     -L1*sin(x+a)-L2*cos(y)-L3/2 ])

V_G   = diff(r_G1A,t)
V_G2  = diff(r_G2B,t)  
V_rack   = diff(r_Gr,t)

# V_G   = diff(r_G1A,t).subs([(diff(x,t,1), xd), (diff(y,t,1), yd)\
#                            ,(diff(z,t,1), zd), (diff(w,t,1), wd)])
# V_G2  = diff(r_G2B,t).subs([(diff(x,t,1), xd), (diff(y,t,1), yd)\
#                            ,(diff(z,t,1), zd), (diff(w,t,1), wd)])   
# V_rack   = diff(r_Gr,t).subs([(diff(x,t,1), xd), (diff(y,t,1), yd)\
#                            ,(diff(z,t,1), zd), (diff(w,t,1), wd)])

display(V_G)
display(V_G2)
display(V_rack)

Matrix([
[ L_1*sin(alpha + delta_1(t))*Derivative(delta_1(t), t) - L_2*cos(phi_1(t))*Derivative(phi_1(t), t)/2],
[-L_1*cos(alpha + delta_1(t))*Derivative(delta_1(t), t) + L_2*sin(phi_1(t))*Derivative(phi_1(t), t)/2]])

Matrix([
[-L_1*sin(alpha - delta_2(t))*Derivative(delta_2(t), t) - L_2*cos(phi_2(t))*Derivative(phi_2(t), t)/2],
[ L_1*cos(alpha - delta_2(t))*Derivative(delta_2(t), t) - L_2*sin(phi_2(t))*Derivative(phi_2(t), t)/2]])

Matrix([
[                                                                                                 0],
[-L_1*cos(alpha + delta_1(t))*Derivative(delta_1(t), t) + L_2*sin(phi_1(t))*Derivative(phi_1(t), t)]])

In [43]:
T=(  I1*diff(x,t)**2    \
   + I2*diff(y,t)**2    \
   + m2*V_G.dot(V_G)    \
   + mr*V_rack.dot(V_rack)    \
   + I1*diff(z,t)**2    \
   + I2*diff(w,t)**2    \
   + m2*V_G2.dot(V_G2)  )/2

# T = T.subs([(diff(x,t,1), xd), (diff(y,t,1), yd)\
#                            ,(diff(z,t,1), zd), (diff(w,t,1), wd)])
T = simplify(T)
display(T)

I_1*Derivative(delta_1(t), t)**2/2 + I_1*Derivative(delta_2(t), t)**2/2 + I_2*Derivative(phi_1(t), t)**2/2 + I_2*Derivative(phi_2(t), t)**2/2 + m_2*(4*L_1**2*Derivative(delta_1(t), t)**2 - 4*L_1*L_2*sin(alpha + delta_1(t) + phi_1(t))*Derivative(delta_1(t), t)*Derivative(phi_1(t), t) + L_2**2*Derivative(phi_1(t), t)**2)/8 + m_2*(4*L_1**2*Derivative(delta_2(t), t)**2 - 4*L_1*L_2*sin(-alpha + delta_2(t) + phi_2(t))*Derivative(delta_2(t), t)*Derivative(phi_2(t), t) + L_2**2*Derivative(phi_2(t), t)**2)/8 + m_r*(L_1*cos(alpha + delta_1(t))*Derivative(delta_1(t), t) - L_2*sin(phi_1(t))*Derivative(phi_1(t), t))**2/2

In [44]:

Eq = Matrix( [diff(diff(T, diff(x, t)), t) - diff(T, x),\
              diff(diff(T, diff(y, t)), t) - diff(T, y),\
              diff(diff(T, diff(z, t)), t) - diff(T, z),\
              diff(diff(T, diff(w, t)), t) - diff(T, w)] )



In [45]:


Eq = Eq.subs([  (diff(x,t,2), xdd),(diff(xd,t,1), xdd),(diff(x,t,1), xd)\
               ,(diff(y,t,2), ydd),(diff(yd,t,1), ydd),(diff(y,t,1), yd)\
               ,(diff(z,t,2), zdd),(diff(zd,t,1), zdd),(diff(z,t,1), zd)\
               ,(diff(w,t,2), wdd),(diff(wd,t,1), wdd),(diff(w,t,1), wd)    ])


# Eq = simplify(Eq)

In [46]:
Eq = expand(Eq)

Lside_factors = [xdd,ydd,xd**2,yd**2,zdd,wdd,zd**2,wd**2]

Lside = zeros(4,8)


for i,eq in enumerate(Eq):
    A = collect(eq,Lside_factors,evaluate=False) 
    for j,f in enumerate(Lside_factors):
        if f in A:
            Lside[i,j] = A[f]
            
Lside_factors = Matrix(Lside_factors)

display(Lside)
display(Lside_factors)

Matrix([
[                                             I_1 + L_1**2*m_2 + L_1**2*m_r*cos(alpha + delta_1(t))**2, -L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)),                                           -L_1**2*m_r*sin(alpha + delta_1(t))*cos(alpha + delta_1(t)), -L_1*L_2*m_2*cos(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*cos(alpha + delta_1(t))*cos(phi_1(t)),                                                  0,                                                  0,                                                  0,                                                  0],
[-L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)),                                                      I_2 + L_2**2*m_2/4 + L_2**2*m_r*sin(phi_1(t))**2, -L_1*L_2*m_2*cos(alpha + delta_1(t) + phi_1(t))/2 + L_1*L_2*m_r*sin(alpha + delta_1(t))*sin(phi_1(t)),                                                            

Matrix([
[  \ddot{\delta}_1(t)],
[    \ddot{\phi}_1(t)],
[\dot{\delta}_1(t)**2],
[  \dot{\phi}_1(t)**2],
[  \ddot{\delta}_2(t)],
[    \ddot{\phi}_2(t)],
[\dot{\delta}_2(t)**2],
[  \dot{\phi}_2(t)**2]])

In [26]:
# constrain Eq

Const_Eq = Matrix([  -d + L2*sin(y) + L1*cos(x+a)
                    ,-d + L2*sin(w) + L1*cos(a-z)
                    ,L1*sin(x+a)+L2*cos(y)+L1*sin(a-z)+L2*cos(w)-D+L3])

# display(CEq)

CEq = diff(Const_Eq,t)

CEq = CEq.subs([    (diff(x,t,1), xd)\
                   ,(diff(y,t,1), yd)\
                   ,(diff(z,t,1), zd)\
                   ,(diff(w,t,1), wd)    ])

display(CEq)

Rside = zeros(4,3+external_inputs_num)

factors = [xd,yd,zd,wd]

for i,ceq in enumerate(CEq):
    A = collect(ceq,factors,evaluate=False)
    
    for j,f in enumerate(factors):
        if f in A:
            Rside[j,i+external_inputs_num] = A[f]
            

Matrix([
[                                                                                   -L_1*\dot{\delta}_1(t)*sin(alpha + delta_1(t)) + L_2*\dot{\phi}_1(t)*cos(phi_1(t))],
[                                                                                    L_1*\dot{\delta}_2(t)*sin(alpha - delta_2(t)) + L_2*\dot{\phi}_2(t)*cos(phi_2(t))],
[L_1*\dot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_1*\dot{\delta}_2(t)*cos(alpha - delta_2(t)) - L_2*\dot{\phi}_1(t)*sin(phi_1(t)) - L_2*\dot{\phi}_2(t)*sin(phi_2(t))]])

In [27]:
# Virtual Work
F = Matrix([0,-1]) 

dw = Matrix([xd , zd , F.dot(V_rack) ]) # *[M1,M2,Fr]

dw = dw.subs([      (diff(x,t,1), xd)\
                   ,(diff(y,t,1), yd)\
                   ,(diff(z,t,1), zd)\
                   ,(diff(w,t,1), wd)    ])
Dw = expand(dw)
display(Dw)

factors = [xd,yd,zd,wd]

for i,dw in enumerate(Dw):
    A = collect(dw,factors,evaluate=False)
    
    for j,f in enumerate(factors):
        if f in A:
            Rside[j,i] = A[f]

display(Rside)

Rside_factors = Matrix([[M1],[M2],[Fr],[lan1],[lan2],[lan3]])
display(Rside_factors)

Matrix([
[                                                                \dot{\delta}_1(t)],
[                                                                \dot{\delta}_2(t)],
[L_1*\dot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_2*\dot{\phi}_1(t)*sin(phi_1(t))]])

Matrix([
[1, 0, L_1*cos(alpha + delta_1(t)), -L_1*sin(alpha + delta_1(t)),                           0,  L_1*cos(alpha + delta_1(t))],
[0, 0,          -L_2*sin(phi_1(t)),            L_2*cos(phi_1(t)),                           0,           -L_2*sin(phi_1(t))],
[0, 1,                           0,                            0, L_1*sin(alpha - delta_2(t)), -L_1*cos(alpha - delta_2(t))],
[0, 0,                           0,                            0,           L_2*cos(phi_2(t)),           -L_2*sin(phi_2(t))]])

Matrix([
[   tau_1],
[   tau_2],
[     F_r],
[lambda_1],
[lambda_2],
[lambda_3]])

In [28]:
# combile All

All = Lside.row_join(-Rside)
All_factors = Lside_factors.col_join(Rside_factors)

display(All)
display(All_factors)

Matrix([
[                                             I_1 + L_1**2*m_2 + L_1**2*m_r*cos(alpha + delta_1(t))**2, -L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)),                                           -L_1**2*m_r*sin(alpha + delta_1(t))*cos(alpha + delta_1(t)), -L_1*L_2*m_2*cos(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*cos(alpha + delta_1(t))*cos(phi_1(t)),                                                  0,                                                  0,                                                  0,                                                  0, -1,  0, -L_1*cos(alpha + delta_1(t)), L_1*sin(alpha + delta_1(t)),                            0, -L_1*cos(alpha + delta_1(t))],
[-L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)),                                                      I_2 + L_2**2*m_2/4 + L_2**2*m_r*sin(phi_1(t))**2, -L_1*L_2*m_2*cos(alpha + delta_1(t)

Matrix([
[  \ddot{\delta}_1(t)],
[    \ddot{\phi}_1(t)],
[\dot{\delta}_1(t)**2],
[  \dot{\phi}_1(t)**2],
[  \ddot{\delta}_2(t)],
[    \ddot{\phi}_2(t)],
[\dot{\delta}_2(t)**2],
[  \dot{\phi}_2(t)**2],
[               tau_1],
[               tau_2],
[                 F_r],
[            lambda_1],
[            lambda_2],
[            lambda_3]])

In [29]:
# a11,a21,a31,a41 = symbols("a_{11},a_{21},a_{31},a_{41},")
# r11,r21,r31,r41 = symbols("r_{11},r_{21},r_{31},r_{41},")
# r12,r22,r32,r42 = symbols("r_{12},r_{22},r_{32},r_{42},")
# r13,r23,r33,r43 = symbols("r_{13},r_{23},r_{33},r_{43},")


# A = zeros(4,4)
# A[0,0] = -a11
# A[1,0] = -a21
# A[2,0] = -a31
# A[3,0] = -a41

# A[0,1] = r11
# A[1,1] = r21
# A[2,1] = 0
# A[3,1] = 0

# A[0,2] = 0
# A[1,2] = 0
# A[2,2] = r32
# A[3,2] = r42

# A[0,3] = r13
# A[1,3] = r23
# A[2,3] = r33
# A[3,3] = r43




# # A[:,1:] = All[:,[11,12,13]]*-1

# display(A)
# A = A.inv()

# DEN = A[0]/(-r21*r32*r43+r21*r33*r42)
# A = A/DEN

# DEN = DEN.subs([(a31,0),(a41,0)])
# A = A.subs([(a31,0),(a41,0)])

# display(DEN)

# display(A)

In [30]:
# solve for F landa1 landa2 landa3
# C = A*[theta_dd landa1 landa2 landa3]'

C = All[:,1:11]
A = All[:,[0,11,12,13]] * -1
A_inv = A.inv()
Eq_X = A_inv*C

C = All[:,0:10]
A = All[:,[10,11,12,13]] * -1
A_inv = A.inv()
Eq_F = A_inv*C

Eq_X = Eq_X[0,:]
Eq_F = Eq_F[0,:]

display(Eq_F)
display(Eq_X)


Matrix([[(-L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)))*sin(alpha + delta_1(t))/(-L_2*sin(alpha + delta_1(t))*sin(phi_1(t)) + L_2*cos(alpha + delta_1(t))*cos(phi_1(t))) + (I_1 + L_1**2*m_2 + L_1**2*m_r*cos(alpha + delta_1(t))**2)*cos(phi_1(t))/(-L_1*sin(alpha + delta_1(t))*sin(phi_1(t)) + L_1*cos(alpha + delta_1(t))*cos(phi_1(t))), (I_2 + L_2**2*m_2/4 + L_2**2*m_r*sin(phi_1(t))**2)*sin(alpha + delta_1(t))/(-L_2*sin(alpha + delta_1(t))*sin(phi_1(t)) + L_2*cos(alpha + delta_1(t))*cos(phi_1(t))) + (-L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)))*cos(phi_1(t))/(-L_1*sin(alpha + delta_1(t))*sin(phi_1(t)) + L_1*cos(alpha + delta_1(t))*cos(phi_1(t))), -L_1**2*m_r*sin(alpha + delta_1(t))*cos(alpha + delta_1(t))*cos(phi_1(t))/(-L_1*sin(alpha + delta_1(t))*sin(phi_1(t)) + L_1*cos(alpha + delta_1(t))*cos(phi_1(t))) + (-L_1*L_2*m_2*cos(alpha + delta_1(t) + phi_1(t))/2 + L_1*L_2*m_r*sin(a

Matrix([[-2*L_1*(I_2 + L_2**2*m_2/4 + L_2**2*m_r*sin(phi_1(t))**2)*sin(alpha + delta_1(t))/(2*I_1*L_2*cos(phi_1(t)) - L_1**2*L_2*m_2*sin(alpha + delta_1(t))*sin(alpha + delta_1(t) + phi_1(t)) + 2*L_1**2*L_2*m_2*cos(phi_1(t)) - 2*L_1**2*L_2*m_r*sin(alpha + delta_1(t))*sin(phi_1(t))*cos(alpha + delta_1(t)) + 2*L_1**2*L_2*m_r*cos(alpha + delta_1(t))**2*cos(phi_1(t))) - 2*(-L_1*L_2*m_2*sin(alpha + delta_1(t) + phi_1(t))/2 - L_1*L_2*m_r*sin(phi_1(t))*cos(alpha + delta_1(t)))*cos(phi_1(t))/(2*I_1*cos(phi_1(t)) - L_1**2*m_2*sin(alpha + delta_1(t))*sin(alpha + delta_1(t) + phi_1(t)) + 2*L_1**2*m_2*cos(phi_1(t)) - 2*L_1**2*m_r*sin(alpha + delta_1(t))*sin(phi_1(t))*cos(alpha + delta_1(t)) + 2*L_1**2*m_r*cos(alpha + delta_1(t))**2*cos(phi_1(t))), 2*L_1**2*m_r*sin(alpha + delta_1(t))*cos(alpha + delta_1(t))*cos(phi_1(t))/(2*I_1*cos(phi_1(t)) - L_1**2*m_2*sin(alpha + delta_1(t))*sin(alpha + delta_1(t) + phi_1(t)) + 2*L_1**2*m_2*cos(phi_1(t)) - 2*L_1**2*m_r*sin(alpha + delta_1(t))*sin(phi_1(t))*cos(

In [31]:
# evaluate x,y,z,w
# C = -D+L3+L1*sin(x+a)+L2*cos(y)
C = Symbol('C')
CEQ3 = Const_Eq[2] + D-L3-L1*sin(x+a)-L2*cos(y)+C

Mat = Matrix(solve([Const_Eq[1], CEQ3 ],[z,w]))

In [32]:
Mat

Matrix([
[     alpha + asin((C + L_2*cos(2*atan((2*L_2*d - sqrt(-C**4 + 2*C**2*L_1**2 + 2*C**2*L_2**2 - 2*C**2*d**2 - L_1**4 + 2*L_1**2*L_2**2 + 2*L_1**2*d**2 - L_2**4 + 2*L_2**2*d**2 - d**4))/(C**2 - 2*C*L_2 - L_1**2 + L_2**2 + d**2))))/L_1), 2*atan((2*L_2*d - sqrt(-C**4 + 2*C**2*L_1**2 + 2*C**2*L_2**2 - 2*C**2*d**2 - L_1**4 + 2*L_1**2*L_2**2 + 2*L_1**2*d**2 - L_2**4 + 2*L_2**2*d**2 - d**4))/(C**2 - 2*C*L_2 - L_1**2 + L_2**2 + d**2))],
[     alpha + asin((C + L_2*cos(2*atan((2*L_2*d + sqrt(-C**4 + 2*C**2*L_1**2 + 2*C**2*L_2**2 - 2*C**2*d**2 - L_1**4 + 2*L_1**2*L_2**2 + 2*L_1**2*d**2 - L_2**4 + 2*L_2**2*d**2 - d**4))/(C**2 - 2*C*L_2 - L_1**2 + L_2**2 + d**2))))/L_1), 2*atan((2*L_2*d + sqrt(-C**4 + 2*C**2*L_1**2 + 2*C**2*L_2**2 - 2*C**2*d**2 - L_1**4 + 2*L_1**2*L_2**2 + 2*L_1**2*d**2 - L_2**4 + 2*L_2**2*d**2 - d**4))/(C**2 - 2*C*L_2 - L_1**2 + L_2**2 + d**2))],
[alpha - asin((C + L_2*cos(2*atan((2*L_2*d - sqrt(-C**4 + 2*C**2*L_1**2 + 2*C**2*L_2**2 - 2*C**2*d**2 - L_1**4 + 2*L_1**2*L_2**

In [33]:
Y_val = Matrix(solve(Const_Eq[0],y))[1,0]
# Z_val = Mat[0,0]
W_val = Mat[0,1]

# Z_val = Z_val.subs([(C, -D+L3+L1*sin(x+a)+L2*cos(y))])
Z_val = a + asin((L2*cos(w)+C)/L1).subs([(C, -D+L3+L1*sin(x+a)+L2*cos(y))])
W_val = W_val.subs([(C, -D+L3+L1*sin(x+a)+L2*cos(y))])

display(Y_val)
display(Z_val)
display(W_val)

-asin((L_1*cos(alpha + delta_1(t)) - d)/L_2)

alpha + asin((-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_2*cos(phi_2(t)) + L_3)/L_1)

2*atan((2*L_2*d - sqrt(-L_1**4 + 2*L_1**2*L_2**2 + 2*L_1**2*d**2 + 2*L_1**2*(-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3)**2 - L_2**4 + 2*L_2**2*d**2 + 2*L_2**2*(-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3)**2 - d**4 - 2*d**2*(-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3)**2 - (-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3)**4))/(-L_1**2 + L_2**2 - 2*L_2*(-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3) + d**2 + (-D + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3)**2))

In [34]:
# C2 = Symbol('C_2')
# CEQ3 = CEq[2] + xd*L1*sin(x)+yd*L2*sin(y)+C2

Mat = solve([CEq[1], CEq[2] ],[zd,wd])

Yd_val = Matrix(solve(CEq[0],yd))[0]
Zd_val = Mat[zd]
Wd_val = Mat[wd]

Zd_val = simplify(Zd_val)
Wd_val = simplify(Wd_val)

display(Yd_val)
display(Zd_val)
display(Wd_val)

L_1*\dot{\delta}_1(t)*sin(alpha + delta_1(t))/(L_2*cos(phi_1(t)))

(L_1*\dot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_2*\dot{\phi}_1(t)*sin(phi_1(t)))*cos(phi_2(t))/(L_1*cos(alpha - delta_2(t) + phi_2(t)))

-(L_1*\dot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_2*\dot{\phi}_1(t)*sin(phi_1(t)))*sin(alpha - delta_2(t))/(L_2*cos(alpha - delta_2(t) + phi_2(t)))

In [35]:
CEq_dd = diff(Const_Eq,t,2)

CEq_dd = CEq_dd.subs([    (diff(x,t,2), xdd)\
                        ,(diff(y,t,2), ydd)\
                        ,(diff(z,t,2), zdd)\
                        ,(diff(w,t,2), wdd)\
                        ,(diff(x,t,1), xd)\
                        ,(diff(y,t,1), yd)\
                        ,(diff(z,t,1), zd)\
                        ,(diff(w,t,1), wd)    ])


Mat = solve([CEq_dd[1], CEq_dd[2]],[zdd,wdd])

Ydd_val = Matrix(solve(CEq_dd[0],ydd))[0]
Zdd_val = Mat[zdd]
Wdd_val = Mat[wdd]

Ydd_val = simplify(Ydd_val)
Zdd_val = simplify(Zdd_val)
Wdd_val = simplify(Wdd_val)

display(Ydd_val)
display(Zdd_val)
display(Wdd_val)

(L_1*(\ddot{\delta}_1(t)*sin(alpha + delta_1(t)) + \dot{\delta}_1(t)**2*cos(alpha + delta_1(t))) + L_2*\dot{\phi}_1(t)**2*sin(phi_1(t)))/(L_2*cos(phi_1(t)))

(L_1*\ddot{\delta}_1(t)*cos(alpha + delta_1(t))*cos(phi_2(t)) - L_1*\dot{\delta}_1(t)**2*sin(alpha + delta_1(t))*cos(phi_2(t)) - L_1*\dot{\delta}_2(t)**2*sin(alpha - delta_2(t) + phi_2(t)) - L_2*\ddot{\phi}_1(t)*sin(phi_1(t))*cos(phi_2(t)) - L_2*\dot{\phi}_1(t)**2*cos(phi_1(t))*cos(phi_2(t)) - L_2*\dot{\phi}_2(t)**2)/(L_1*cos(alpha - delta_2(t) + phi_2(t)))

(-L_1*\ddot{\delta}_1(t)*sin(alpha - delta_2(t))*cos(alpha + delta_1(t)) + L_1*\dot{\delta}_1(t)**2*sin(alpha + delta_1(t))*sin(alpha - delta_2(t)) + L_1*\dot{\delta}_2(t)**2 + L_2*\ddot{\phi}_1(t)*sin(alpha - delta_2(t))*sin(phi_1(t)) + L_2*\dot{\phi}_1(t)**2*sin(alpha - delta_2(t))*cos(phi_1(t)) + L_2*\dot{\phi}_2(t)**2*sin(alpha - delta_2(t) + phi_2(t)))/(L_2*cos(alpha - delta_2(t) + phi_2(t)))

In [51]:
# calculate x xd xdd from dx dxd dxdd
dr = Symbol('\Delta{r}')
drd = Symbol('\dot{r}')
drdd = Symbol('\ddot{r}')

dr2x_val = zeros(3,1)

Delta_eq0 = -r_Gr[1]-dr-D/2
Mat = Matrix(solve(Delta_eq0,x))
dr2x_val[0] = Mat[0]

Delta_eq1 = diff(Delta_eq0,t,1).subs([(diff(x,t,1), xd),(diff(y,t,1), yd)]) - drd
Mat = Matrix(solve(Delta_eq1,xd))
dr2x_val[1] = Mat[0]

Delta_eq2 = diff(Delta_eq0,t,2).subs([(diff(x,t,2), xdd),(diff(y,t,2), ydd),(diff(x,t,1), xd),(diff(y,t,1), yd)]) - drdd
Mat = Matrix(solve(Delta_eq2,xdd))
dr2x_val[2] = Mat[0]

display(dr2x_val)

Matrix([
[                                                                                                       -alpha + asin((D/2 - L_2*cos(phi_1(t)) - L_3/2 + \Delta{r})/L_1)],
[                                                                                            (L_2*\dot{\phi}_1(t)*sin(phi_1(t)) + \dot{r})/(L_1*cos(alpha + delta_1(t)))],
[(L_1*\dot{\delta}_1(t)**2*sin(alpha + delta_1(t)) + L_2*\ddot{\phi}_1(t)*sin(phi_1(t)) + L_2*\dot{\phi}_1(t)**2*cos(phi_1(t)) + \ddot{r})/(L_1*cos(alpha + delta_1(t)))]])

Matrix([
[                                                                                                       -alpha + asin((D/2 - L_2*cos(phi_1(t)) - L_3/2 + \Delta{r})/L_1)],
[                                                                                            (L_2*\dot{\phi}_1(t)*sin(phi_1(t)) + \dot{r})/(L_1*cos(alpha + delta_1(t)))],
[(L_1*\dot{\delta}_1(t)**2*sin(alpha + delta_1(t)) + L_2*\ddot{\phi}_1(t)*sin(phi_1(t)) + L_2*\dot{\phi}_1(t)**2*cos(phi_1(t)) + \ddot{r})/(L_1*cos(alpha + delta_1(t)))]])

In [52]:
x2dr_val = zeros(3,1)

x2dr_val[0] = -r_Gr[1] - D/2
x2dr_val[1] = diff(x2dr_val[0],t,1).subs([(diff(x,t,1), xd),(diff(y,t,1), yd)])
x2dr_val[2] = diff(x2dr_val[0],t,2).subs([(diff(x,t,2), xdd),(diff(y,t,2), ydd),(diff(x,t,1), xd),(diff(y,t,1), yd)])

display(x2dr_val)

Matrix([
[                                                                                                               -D/2 + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3/2],
[                                                                                            L_1*\dot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_2*\dot{\phi}_1(t)*sin(phi_1(t))],
[L_1*\ddot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_1*\dot{\delta}_1(t)**2*sin(alpha + delta_1(t)) - L_2*\ddot{\phi}_1(t)*sin(phi_1(t)) - L_2*\dot{\phi}_1(t)**2*cos(phi_1(t))]])

Matrix([
[                                                                                                               -D/2 + L_1*sin(alpha + delta_1(t)) + L_2*cos(phi_1(t)) + L_3/2],
[                                                                                            L_1*\dot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_2*\dot{\phi}_1(t)*sin(phi_1(t))],
[L_1*\ddot{\delta}_1(t)*cos(alpha + delta_1(t)) - L_1*\dot{\delta}_1(t)**2*sin(alpha + delta_1(t)) - L_2*\ddot{\phi}_1(t)*sin(phi_1(t)) - L_2*\dot{\phi}_1(t)**2*cos(phi_1(t))]])

In [50]:
print_latex( dr2x_val[0] )
print_latex( dr2x_val[1] )
print_latex( dr2x_val[2] )

print_latex( x2dr_val[0] )
print_latex( x2dr_val[1] )
print_latex( x2dr_val[2] )

- \alpha + \operatorname{asin}{\left(\frac{\frac{D}{2} - L_{2} \cos{\left(\phi_{1}{\left(t \right)} \right)} - \frac{L_{3}}{2} + \Delta{r}}{L_{1}} \right)}
\frac{L_{2} \dot{\phi}_1{\left(t \right)} \sin{\left(\phi_{1}{\left(t \right)} \right)} + \dot{r}}{L_{1} \cos{\left(\alpha + \delta_{1}{\left(t \right)} \right)}}
\frac{L_{1} \dot{\delta}_1^{2}{\left(t \right)} \sin{\left(\alpha + \delta_{1}{\left(t \right)} \right)} + L_{2} \ddot{\phi}_1{\left(t \right)} \sin{\left(\phi_{1}{\left(t \right)} \right)} + L_{2} \dot{\phi}_1^{2}{\left(t \right)} \cos{\left(\phi_{1}{\left(t \right)} \right)} + \ddot{r}}{L_{1} \cos{\left(\alpha + \delta_{1}{\left(t \right)} \right)}}
- \frac{D}{2} + L_{1} \sin{\left(\alpha + \delta_{1}{\left(t \right)} \right)} + L_{2} \cos{\left(\phi_{1}{\left(t \right)} \right)} + \frac{L_{3}}{2}
L_{1} \dot{\delta}_1{\left(t \right)} \cos{\left(\alpha + \delta_{1}{\left(t \right)} \right)} - L_{2} \dot{\phi}_1{\left(t \right)} \sin{\left(\phi_{1}{\left(t \right)} \right

In [38]:


# Eq_F = simplify(Eq_F[0].subs(INIT))
# Eq_X = simplify(Eq_X[0].subs(INIT))

# Y_val = simplify(Y_val.subs(INIT))
# Z_val = simplify(Z_val.subs(INIT))
# W_val = simplify(W_val.subs(INIT))
# Yd_val = simplify(Yd_val.subs(INIT))
# Zd_val = simplify(Zd_val.subs(INIT))
# Wd_val = simplify(Wd_val.subs(INIT))
# Ydd_val = simplify(Ydd_val.subs(INIT))
# Zdd_val = simplify(Zdd_val.subs(INIT))
# Wdd_val = simplify(Wdd_val.subs(INIT))

# dr2x_val = simplify(dr2x_val.subs(INIT))
# x2dr_val = simplify(x2dr_val.subs(INIT))


# Eq_F = (Eq_F.subs(INIT))
# Eq_X = (Eq_X.subs(INIT))

# Y_val = (Y_val.subs(INIT))
# Z_val = (Z_val.subs(INIT))
# W_val = (W_val.subs(INIT))
# Yd_val = (Yd_val.subs(INIT))
# Zd_val = (Zd_val.subs(INIT))
# Wd_val = (Wd_val.subs(INIT))
# Ydd_val = (Ydd_val.subs(INIT))
# Zdd_val = (Zdd_val.subs(INIT))
# Wdd_val = (Wdd_val.subs(INIT))

# dr2x_val = (dr2x_val.subs(INIT))
# x2dr_val = (x2dr_val.subs(INIT))


In [39]:
MAP = { dr: Symbol('dr'),drd: Symbol('drd'),drdd: Symbol('drdd'),\
        x: Symbol('x'),    y: Symbol('y'),    z: Symbol('z'),    w: Symbol('w'), \
       xd: Symbol('xd'),  yd: Symbol('yd'),  zd: Symbol('zdd'), wd: Symbol('wd'), \
      xdd: Symbol('xdd'),ydd: Symbol('ydd'),zdd: Symbol('zdd'),wdd: Symbol('wdd')}

dr = Symbol('\Delta{r}')
drd = Symbol('\dot{r}')
drdd = Symbol('\ddot{r}')
x = Function("delta_1")(t) 
y = Function("phi_1")(t)
z = Function("delta_2")(t) 
w = Function("phi_2")(t)
xd = Function("\dot{\\delta}_1")(t)
yd = Function("\dot{\\phi}_1")(t)
zd = Function("\dot{\\delta}_2")(t)
wd = Function("\dot{\\phi}_2")(t)
xdd = Function("\ddot{\\delta}_1")(t)
ydd = Function("\ddot{\\phi}_1")(t)
zdd = Function("\ddot{\\delta}_2")(t)
wdd = Function("\ddot{\\phi}_2")(t)

dr = MAP[dr]
drd = MAP[drd]
drdd = MAP[drdd]
x = MAP[x]
y = MAP[y]
z = MAP[z]
w = MAP[w]
xd = MAP[xd]
yd = MAP[yd]
zd = MAP[zd]
wd = MAP[wd]
xdd = MAP[xdd]
ydd = MAP[ydd]
zdd = MAP[zdd]
wdd = MAP[wdd]



F   = lambdify([x, y, z, w], Eq_F.xreplace(MAP), "numpy")
Xdd = lambdify([x, y, z, w], Eq_X.xreplace(MAP), "numpy")

Y = lambdify( x, Y_val.xreplace(MAP), "numpy")
Z = lambdify([x,y,w], Z_val.xreplace(MAP), "numpy")
W = lambdify([x,y], W_val.xreplace(MAP), "numpy")

Yd = lambdify([x,xd,y], Yd_val.xreplace(MAP), "numpy")
Zd = lambdify([x,xd,y,yd,z,w], Zd_val.xreplace(MAP), "numpy")
Wd = lambdify([x,xd,y,yd,z,w], Wd_val.xreplace(MAP), "numpy")

Ydd = lambdify([x,xd,xdd,y,yd], Ydd_val.xreplace(MAP), "numpy")
Zdd = lambdify([x,xd,xdd,y,yd,ydd,z,zd,w,wd], Zdd_val.xreplace(MAP), "numpy")
Wdd = lambdify([x,xd,xdd,y,yd,ydd,z,zd,w,wd], Wdd_val.xreplace(MAP), "numpy")

dr2x_0 = lambdify([dr,y], dr2x_val[0].xreplace(MAP), "numpy")
dr2x_1 = lambdify([drd,x,y,yd], dr2x_val[1].xreplace(MAP), "numpy")
dr2x_2 = lambdify([drdd,x,xd,y,yd,ydd], dr2x_val[2].xreplace(MAP), "numpy")

x2dr = lambdify([x,xd,xdd,y,yd,ydd], x2dr_val.xreplace(MAP), "numpy")

In [40]:
import inspect


STR = "from numpy import array,sin,cos,tan,arcsin,arccos,arctan,sqrt,pi"+'\n\n\n'\
      +"alpha=17.5*pi/180\nL_1=0.14\nL_2=0.5\nL_3=0.5246808715635702\nd=0.2\nD=1.6\n" \
      +"m_2=1.12\nm_r=1.17\nI_1=0.7761601605732\nI_2=0.024676 \n\n\n"\
      +inspect.getsource(F)+'\n\n\n'\
      +inspect.getsource(Xdd)+'\n\n\n'\
      +inspect.getsource(Y)+'\n\n\n'\
      +inspect.getsource(Z)+'\n\n\n'\
      +inspect.getsource(W)+'\n\n\n'\
      +inspect.getsource(Yd)+'\n\n\n'\
      +inspect.getsource(Zd)+'\n\n\n'\
      +inspect.getsource(Wd)+'\n\n\n'\
      +inspect.getsource(Ydd)+'\n\n\n'\
      +inspect.getsource(Zdd)+'\n\n\n'\
      +inspect.getsource(Wdd)+'\n\n\n'\
      +inspect.getsource(dr2x_0)+'\n\n\n'\
      +inspect.getsource(dr2x_1)+'\n\n\n'\
      +inspect.getsource(dr2x_2)+'\n\n\n'\
      +inspect.getsource(x2dr)+'\n\n\n'

names = ['F','Xdd','Y','Z','W','Yd','Zd','Wd','Ydd','Zdd','Wdd','dr_to_x_0','dr_to_x_1','dr_to_x_2','x_to_dr']
for i in range(len(names)):
    STR = STR.replace('_lambdifygenerated',names[i],1)

text_file = open("Output_functions.py", "w")
text_file.write(STR)
text_file.close()